# Overview

PyLag is an offline particle tracking model. The model expects as inputs time independent and/or dependent variables that describe the state of a given fluid. These may be measured quantities or the predictions of an analytical or numerical model. Using these, the model computes Lagrangian trajectories for particles released into the fluid at a particular point in time and space. The model is primarily aimed at marine applications, but is intended to be versatile enough to be extended to other contexts; for example, studies of atmospheric dispersion.

PyLag was created with the aim to make available a particle tracking model that is a) fast to run, b) easy to use, c) extensible and d) flexible. The model is written in a mixture of [Python](http://www.python.org), [Cython](http://www.cython.org) and C++.

## Code structure

A simplified Unified Modelling Language (UML) Class Diagram for the model is shown in Figure 1. Functions that perform numerically intensive tasks have been implemented in Cython or C++ (dark grey boxes), while those that perform less numerically intensive tasks have been implemented in Python (pale grey boxes). The division is intended to combine the ease of use of Python with the efficiency gains offered by Cython and C++. During compilation, Cython source files are parsed and translated into optimized C++ code, which is them compiled into extension modules that can be loaded at runtime by the Python Interpreter.

![Code structure](figures/fig_code_structure_a.png)
Figure 1: UML Class Diagram.

The model currently includes direct support for the General Ocean Turbulence Model (GOTM), the Finite Volume Community Ocean Model (FVCOM) and data defined on a regular Arakawa-a grid (not shown in Figure 1). GOTM is a one-dimensional, relocatable water column model that includes state-of-the-art descriptions of vertical turbulent mixing (Umlauf et al 2008). FVCOM is a prognostic, unstructured-grid, finite-volume, free-surface, 3D primitive equation coastal ocean circulation model (Chen 2003). Each input source has associated with it a derived `DataReader` class that inherits from a base class `DataReader` (see Figure 1). The primary purpose of such objects is to compute and provide access to the value of a given variable (e.g. the vertical turbulent eddy diffusivity) at a particular point in space and time. For gridded input data, this process is facilitated through interpolation, while taking into account the structure of the underlying model grid. Figures 1B and 1C give examples of the types of input generated by GOTM and FVCOM respectively: the former shows a hovemoller diagram of the simulated vertical turbulent eddy diffusivity for heat at a location in the Western English Channel; the latter a map of the surface horizontal turbulent eddy diffusivity field near to the coastal city of Plymouth, UK. The code has been designed in a way that makes it possible to easily extend the model to read in data defined on different types of input grid. This can be achieved by sub-classing `DataReader`. 

Given input data, PyLag can compute particle trajectories using one of the different numerical schemes implemented in the model code. In order to support numerical methods that employ some form of operator splitting in which advection and diffusion are handled separately, two different families of object have been implemented in the code. Objects that belong to the first family control the integration process, and may or may not implement some form of operator splitting. They inherit their interface from the base class `NumMethod`. Two derived classes have been implemented: `StdNumMethod` and `OSONumMethod`. The former does not employ operator splitting, while the latter implements a basic form of operator splitting. Objects from the second family represent iterative methods that have no awareness of whether operator splitting is being used: they simply compute position deltas, and return these to the caller. They inherit their interface from the base class `ItMethod`. Several derived classes have been implemented: some of these implement pure deterministic solution methods that should be used for advection-only problems; others implement purely stochastic solution methods that should be used for diffusion-only problems; while others implement solution methods that are suitable for problems involving both diffusion and advection.

The testing of such models is facilitated through the inclusion of various mock classes that are derived from `DataReader`. Mock classes typically encode analytic functions that describe the spatio-temporal variability of a given field variable or variables (e.g. the velocity field). This removes the dependency on external input data. Figure 1A shows an example of a vertical turbulent eddy diffusivity profile computed using an analytic formula. The same profile is used in one of PyLag's [examples](../examples/vertical_mixing_analytic.ipynb) to test for the Well Mixed Condition.

## Supported sources of input data

PyLag currently includes direct support for the following types of input data:

* General Ocean Turbulence Model (GOTM)
* Finite Volume Communitty Ocean Model (FVCOM)
* Regularly gridded data on an Arakawa A-grid (Arakawa-A)

## References

Chen,  C.  H.  Liu,  R.  C.  Beardsley,  2003.  An  unstructured,  finite-volume,  three-dimensio nal, primitive  equation  ocean  model:  application  to  coastal  ocean  and  estuaries.J.  Atm.  &Oceanic Tech., 20, 159-186

Umlauf, L. Burchard, H., 2004. Second-order turbulence closure models for geophysical boundary layers. A review of recent work. Continental Shelf Research 25(7):795-827. DOI: 10.1016/j.csr.2004.08.004